# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, np.float64(1.2292233916655595)),
 (1, np.float64(1.2292233916655595)),
 (2, np.float64(1.2292233916655595)),
 (3, np.float64(1.2292233916655595)),
 (4, np.float64(1.2292233916655595))]

## Inverted index 

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('it', 18), ('what', 10)]),
 (1, [('a', 2), ('banana', 2), ('is', 18)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.02654550487505869)),
   (None, np.float64(0.03862526661616139)),
   (None, np.float64(0.10549359288996352)),
   (None, np.float64(0.1611962634643116)),
   (None, np.float64(0.16954401915056672)),
   (None, np.float64(0.1814631191213245)),
   (None, np.float64(0.18486161521664557)),
   (None, np.float64(0.23901401065889583)),
   (None, np.float64(0.24285176474674508)),
   (None, np.float64(0.2833563367034354)),
   (None, np.float64(0.31519416470099026)),
   (None, np.float64(0.3152020871371831)),
   (None, np.float64(0.34077052191022783)),
   (None, np.float64(0.37032264987078056)),
   (None, np.float64(0.3991618911526503)),
   (None, np.float64(0.42460903480964496)),
   (None, np.float64(0.43486539605510766))]),
 (1,
  [(None, np.float64(0.5113481396154398)),
   (None, np.float64(0.5193825481879765)),
   (None, np.float64(0.548220616181598)),
   (None, np.float64(0.562144064519448)),
   (None, np.float64(0.6013494515263329)),
   (None, np.float64(0.612939114

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
from random import randint
from functools import reduce

def RECORDREADER(count, part_size):
  data = []
  for i in range(count):
    data.append(randint(-100, 100))

  for i in range(0, count, part_size):
    yield data[i:i+part_size]

def MAP(list_nums):
  return max(list_nums)

def REDUCE(a, b):
  return max(a, b)

record_reader_list = list(RECORDREADER(17, 5))

full_list_nums = [num for part_list in record_reader_list for num in part_list]
print("List:", full_list_nums)

print("Parts list:", record_reader_list)

map_list = list(map(MAP, record_reader_list))
print("Local max:", map_list)

print("Max:", reduce(REDUCE, map_list))


List: [-18, 98, -10, -35, -47, 96, -49, -9, -90, 99, 24, 15, -69, 81, 38, 53, -100]
Parts list: [[-18, 98, -10, -35, -47], [96, -49, -9, -90, 99], [24, 15, -69, 81, 38], [53, -100]]
Local max: [98, 99, 81, 53]
Max: 99


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
from random import randint

def RECORDREADER(count, part_size):
  data = []
  for i in range(count):
    data.append(randint(0, 10))
      
  for i in range(0, count, part_size):
    yield data[i:i+part_size]

def MAP(list_nums):
  return (sum(list_nums), len(list_nums))

def REDUCE(a, b):
  total_sum = a[0] + b[0]
  total_count = a[1] + b[1]
  return (total_sum, total_count)

data_parts = list(RECORDREADER(20, 5))
data = [nums for data_part in data_parts for nums in data_part]
print(data)

map_list = list(map(MAP, data_parts))

print("\nMAP result:")
for value in map_list:
  print(value)

total_sum, total_count = reduce(REDUCE, map_list)

print("Арифметическое среднее (REDUCE)", total_sum / total_count)


[8, 0, 2, 4, 10, 3, 4, 9, 5, 2, 7, 2, 0, 7, 9, 7, 1, 6, 1, 6]

MAP result:
(24, 5)
(23, 5)
(25, 5)
(21, 5)
Арифметическое среднее (REDUCE) 4.65


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
from random import randint
from functools import reduce

def RECORDREADER(count):
  data = []
  for i in range(count):
    data.append(randint(0, 10))
  return data

def MAP(num):
  key = num % 3  
  return key, (num, 1)

def REDUCE(a, b):
  total_sum = a[0] + b[0]
  total_count = a[1] + b[1]
  return total_sum, total_count

def groupByKey(mapped_data):
  mapped_data.sort(key=lambda x: x[0])  #Сортировка по ключу
  grouped_data = {}

  for key, value in mapped_data:
    if key not in grouped_data:
      grouped_data[key] = []
    grouped_data[key].append(value)

  return grouped_data.items()

data = RECORDREADER(20)
print("List:", data)

map_list = list(map(MAP, data))
print("\nMAP result:")
for value in map_list:
  print(value)

grouped_data = groupByKey(map_list)
print("\nGrouped data:")
for key, values in grouped_data:
  print(f"Key {key}: {values}")

reduced_values = []
for key, values in grouped_data:
  reduced_result = reduce(REDUCE, values)
  reduced_values.append((key, reduced_result))

group_result = {}
for key, (total_sum, total_count) in reduced_values:
  group_result[key] = total_sum / total_count
  
print("\nАрифметическое среднее в каждой группе:")
for key, value in group_result.items():
  print(f"Key {key}: {value}")

values_list = []
for _, values in reduced_values:
  values_list.append(values)

total_sum, total_count = reduce(REDUCE, values_list)

print("\nАрифметическое среднее:", total_sum / total_count)


List: [0, 2, 10, 9, 4, 4, 1, 10, 0, 4, 3, 1, 8, 4, 7, 9, 4, 4, 8, 10]

MAP result:
(0, (0, 1))
(2, (2, 1))
(1, (10, 1))
(0, (9, 1))
(1, (4, 1))
(1, (4, 1))
(1, (1, 1))
(1, (10, 1))
(0, (0, 1))
(1, (4, 1))
(0, (3, 1))
(1, (1, 1))
(2, (8, 1))
(1, (4, 1))
(1, (7, 1))
(0, (9, 1))
(1, (4, 1))
(1, (4, 1))
(2, (8, 1))
(1, (10, 1))

Grouped data:
Key 0: [(0, 1), (9, 1), (0, 1), (3, 1), (9, 1)]
Key 1: [(10, 1), (4, 1), (4, 1), (1, 1), (10, 1), (4, 1), (1, 1), (4, 1), (7, 1), (4, 1), (4, 1), (10, 1)]
Key 2: [(2, 1), (8, 1), (8, 1)]

Арифметическое среднее в каждой группе:
Key 0: 4.2
Key 1: 5.25
Key 2: 6.0

Арифметическое среднее: 5.1


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [24]:
from typing import Iterator
import numpy as np

def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  grouped_data = {}
  for key, value in iterable:
    grouped_data.setdefault(key, []).append(value)
  return list(grouped_data.items())

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for key, value in map_partition:
      p = partitions[PARTITIONER(key)]
      p.setdefault(key, []).append(value)
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER is not None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group),
                                                                                  reduce_partition[1]))), reduce_partitions)
  total_values_sent = sum([len(vs) for (k, vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])

  print(f"{total_values_sent} key-value pairs were sent over a network.")
  return reduce_outputs


d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a dog"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid, lineid), line)
  split_size = int(np.ceil(len(documents) / maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i + split_size])

#Разбивает строку на слова и возвращает пары (word, None)
def MAP(docId: str, line: str):
  for word in line.split(" "):
    if word:
      yield (word, None)

#Возвращает ключ (слово)
def REDUCE(key: str, _):
  yield key

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

50 key-value pairs were sent over a network.


[(0, ['dog', 'it', 'what']), (1, ['a', 'is'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [25]:
import random

def RECORDREADER(count):
  data = []
  for _ in range(count):
    data.append((random.randint(0, 100), random.randint(0, 100)))
  return data

def MAP(list_tuples):
  return [(t, t) for t in list_tuples if C(t)]

def REDUCE(map_list):
  return [t for t, _ in map_list]

def C(t):
  return t[0] % 2 == 0 and t[1] % 2 == 0

record = RECORDREADER(10)

mapped = MAP(record)
reduced = REDUCE(mapped)

print("Исходные данные:", record)
print("MAP:", mapped)
print("REDUCE:", reduced)


Исходные данные: [(56, 32), (33, 57), (19, 69), (54, 84), (28, 42), (93, 85), (19, 48), (41, 78), (1, 25), (38, 44)]
MAP: [((56, 32), (56, 32)), ((54, 84), (54, 84)), ((28, 42), (28, 42)), ((38, 44), (38, 44))]
REDUCE: [(56, 32), (54, 84), (28, 42), (38, 44)]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [26]:
import random

def RECORDREADER(count):
  data = []
  for _ in range(count):
    data.append((random.randint(0, 100), random.randint(0, 100)))
  return data

#Проекция на множество S
def MAP(t): 
  S = {10, 20, 30, 40, 50}
  res = tuple(el for el in t if el in S)
  return (res, res) if res else None

def REDUCE(key, values):
  return (key, key)


def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

# Генерация данных
record = RECORDREADER(30)
print("RECORDREADER:", record)

# Применение MAP
mapped = [MAP(x) for x in record if MAP(x) is not None]
print("MAP:", mapped)

# Группировка ключей
shuffle_output = list(group_by_key(mapped))
print("Shuffle:", shuffle_output)

# Применение REDUCE
reduced = [REDUCE(k, v) for k, v in shuffle_output]
print("REDUCE:", reduced)


RECORDREADER: [(33, 63), (74, 17), (97, 50), (13, 71), (85, 76), (38, 42), (83, 57), (60, 56), (11, 13), (30, 70), (28, 91), (70, 53), (52, 67), (55, 45), (84, 80), (76, 7), (34, 88), (7, 88), (83, 17), (86, 17), (94, 44), (91, 16), (94, 55), (65, 8), (47, 79), (70, 5), (49, 58), (92, 48), (50, 39), (44, 93)]
MAP: [((50,), (50,)), ((30,), (30,)), ((50,), (50,))]
Shuffle: [((50,), [(50,), (50,)]), ((30,), [(30,)])]
REDUCE: [((50,), (50,)), ((30,), (30,))]


### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [27]:
import random

def RECORDREADER(count):
  data = []
  for _ in range(count):
    data.append((random.randint(0, 100), random.randint(0, 100)))
  return data

def MAP(t):
  return (t, t)

def REDUCE(key):
  return (key, key)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(30)
print("RECORDREADER:", record)

mapped = list(map(lambda x: MAP(x), record))
print("MAP output:", mapped)

shuffle_output = list(group_by_key(mapped))
print("Shuffle output:", shuffle_output)

reduced = list(map(lambda x: REDUCE(x[0]), shuffle_output))
print("Reduce output:", reduced)

RECORDREADER: [(19, 52), (8, 52), (50, 25), (57, 45), (31, 29), (59, 10), (77, 41), (67, 67), (54, 76), (78, 80), (67, 64), (64, 3), (77, 86), (63, 4), (21, 58), (70, 4), (4, 86), (79, 32), (19, 13), (18, 15), (75, 39), (28, 26), (50, 5), (58, 96), (71, 74), (97, 14), (53, 23), (61, 52), (26, 69), (10, 56)]
MAP output: [((19, 52), (19, 52)), ((8, 52), (8, 52)), ((50, 25), (50, 25)), ((57, 45), (57, 45)), ((31, 29), (31, 29)), ((59, 10), (59, 10)), ((77, 41), (77, 41)), ((67, 67), (67, 67)), ((54, 76), (54, 76)), ((78, 80), (78, 80)), ((67, 64), (67, 64)), ((64, 3), (64, 3)), ((77, 86), (77, 86)), ((63, 4), (63, 4)), ((21, 58), (21, 58)), ((70, 4), (70, 4)), ((4, 86), (4, 86)), ((79, 32), (79, 32)), ((19, 13), (19, 13)), ((18, 15), (18, 15)), ((75, 39), (75, 39)), ((28, 26), (28, 26)), ((50, 5), (50, 5)), ((58, 96), (58, 96)), ((71, 74), (71, 74)), ((97, 14), (97, 14)), ((53, 23), (53, 23)), ((61, 52), (61, 52)), ((26, 69), (26, 69)), ((10, 56), (10, 56))]
Shuffle output: [((19, 52), [(

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [28]:
import random

def RECORDREADER(count):
  data = []
  for _ in range(count):
    data.append((random.randint(0, 5), random.randint(0, 5)))
  return data

def MAP(t):
  return (t, t)

def REDUCE(key, values):
  if len(values) == 2:
    return (key, key)
  return None

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print("RECORDREADER:", record)

mapped = list(map(lambda x: MAP(x), record))
print("MAP:", mapped)

shuffle_output = group_by_key(mapped)
shuffle_output = list(shuffle_output)
print("Shuffle output:", shuffle_output)

reduce_output = [val for val in map(lambda x: REDUCE(*x), shuffle_output) if val is not None]
print("REDUCE:", reduce_output)

RECORDREADER: [(3, 5), (3, 3), (1, 5), (2, 1), (1, 3), (1, 0), (5, 1), (1, 3), (5, 0), (2, 2), (1, 4), (1, 2), (4, 4), (2, 2), (1, 0), (1, 5), (4, 5), (5, 1), (5, 4), (2, 0), (1, 1), (1, 3), (2, 0), (4, 3), (1, 0), (5, 5), (3, 5), (0, 4), (2, 0), (1, 0), (2, 0), (5, 1), (4, 1), (5, 0), (1, 3), (2, 4), (4, 4), (0, 1), (1, 1), (5, 1), (5, 5), (2, 3), (5, 4), (2, 2), (0, 2), (4, 5), (0, 4), (1, 0), (0, 0), (3, 3), (2, 5), (3, 1), (1, 0), (0, 0), (4, 5), (5, 5), (1, 5), (5, 2), (1, 3), (5, 3), (1, 4), (4, 1), (2, 0), (2, 2), (4, 2), (5, 0), (3, 1), (4, 2), (1, 2), (2, 5), (0, 5), (0, 0), (4, 3), (3, 4), (0, 4), (5, 3), (0, 0), (0, 1), (4, 5), (3, 4), (5, 5), (4, 5), (4, 5), (3, 3), (2, 0), (0, 0), (3, 3), (3, 4), (3, 5), (4, 4), (1, 3), (2, 5), (0, 4), (2, 5), (0, 0), (3, 2), (3, 2), (3, 0), (3, 2), (0, 0)]
MAP: [((3, 5), (3, 5)), ((3, 3), (3, 3)), ((1, 5), (1, 5)), ((2, 1), (2, 1)), ((1, 3), (1, 3)), ((1, 0), (1, 0)), ((5, 1), (5, 1)), ((1, 3), (1, 3)), ((5, 0), (5, 0)), ((2, 2), (2, 2)),

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [29]:
import random
from typing import Iterator

def RECORDREADER(count, tuple_size):
  records = []
  for _ in range(count):
    data = tuple((random.randint(0, 3), random.randint(0, 3)) for _ in range(tuple_size))
    rel_id = RS_list[random.randint(0, len(RS_list) - 1)]
    records.append((data, rel_id))
  return records

def MAP(t):
  return t 

def REDUCE(key, values: Iterator[int]):
  if values == [RS_list[0]]:
    return (key, key)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

RS_list = [1, 2]

record = RECORDREADER(100, 2)

mapped = list(map(MAP, record))
print("MAP:", mapped)

shuffle_output = group_by_key(mapped)
shuffle_output = list(shuffle_output)
print("Shuffle output:", shuffle_output)

reduced = [val[0] for val in list(map(lambda x: REDUCE(*x), shuffle_output)) if val is not None]
print("REDUCE:", reduced)

MAP: [(((2, 3), (3, 2)), 2), (((1, 3), (1, 1)), 2), (((0, 1), (1, 3)), 1), (((2, 1), (2, 1)), 2), (((3, 2), (1, 2)), 2), (((3, 0), (3, 2)), 1), (((0, 0), (3, 2)), 1), (((1, 2), (1, 0)), 2), (((2, 3), (2, 0)), 1), (((3, 3), (2, 2)), 2), (((0, 0), (1, 1)), 1), (((0, 2), (2, 3)), 2), (((2, 2), (2, 1)), 2), (((3, 2), (2, 2)), 2), (((0, 2), (3, 0)), 2), (((2, 1), (0, 0)), 2), (((3, 2), (1, 3)), 1), (((3, 0), (3, 3)), 1), (((3, 3), (0, 2)), 1), (((3, 2), (3, 1)), 2), (((2, 2), (1, 1)), 1), (((0, 0), (0, 3)), 2), (((1, 3), (1, 2)), 1), (((3, 2), (1, 2)), 2), (((1, 1), (3, 2)), 2), (((0, 3), (1, 1)), 2), (((2, 2), (3, 2)), 2), (((0, 3), (1, 1)), 2), (((2, 1), (2, 0)), 1), (((0, 0), (2, 2)), 1), (((3, 1), (1, 0)), 2), (((2, 3), (2, 1)), 2), (((2, 3), (1, 0)), 1), (((1, 1), (1, 2)), 1), (((0, 3), (3, 2)), 2), (((2, 1), (2, 2)), 2), (((0, 3), (3, 2)), 1), (((3, 0), (3, 0)), 2), (((1, 1), (0, 3)), 2), (((3, 1), (2, 1)), 1), (((2, 1), (0, 0)), 2), (((1, 2), (1, 0)), 1), (((3, 1), (0, 1)), 1), (((0,

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [30]:
import random

def RECORDREADER(count):
  records = []
  for _ in range(count):
    data = (random.randint(0, 3), random.randint(0, 3))
    rel_id = RS_list[random.randint(0, len(RS_list) - 1)]
    records.append((data, rel_id))
  return records

def MAP(t):
  if t[1] == RS_list[0]:
    return (t[0][1], (t[1], t[0][0]))
  else:
    return (t[0][0], (t[1], t[0][1]))

def REDUCE(key, values):
  res = []
  for v in values:
    res.append((v[0], key, v[1]))
  return res

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

RS_list = [1, 2]

record = RECORDREADER(100)

mapped = list(map(MAP, record))
print("MAP:", mapped)

shuffle_output = group_by_key(mapped)
shuffle_output = list(shuffle_output)
print("Shuffle output:", shuffle_output)

reduced = [val for val in list(map(lambda x: REDUCE(*x), shuffle_output)) if val is not None]
print("REDUCE:", reduced)


MAP: [(3, (1, 2)), (2, (2, 0)), (0, (1, 0)), (0, (1, 3)), (0, (2, 3)), (0, (1, 0)), (3, (1, 0)), (1, (2, 3)), (1, (1, 0)), (1, (2, 0)), (1, (2, 1)), (2, (2, 2)), (1, (1, 2)), (3, (2, 1)), (1, (2, 3)), (3, (1, 2)), (1, (1, 3)), (2, (2, 1)), (2, (1, 1)), (0, (1, 2)), (2, (2, 1)), (3, (2, 0)), (0, (2, 3)), (1, (2, 2)), (1, (2, 2)), (2, (1, 2)), (0, (2, 1)), (0, (1, 2)), (1, (2, 1)), (3, (1, 3)), (3, (2, 3)), (1, (2, 2)), (1, (2, 1)), (0, (1, 3)), (1, (2, 0)), (0, (1, 1)), (2, (1, 3)), (2, (1, 3)), (3, (1, 3)), (0, (1, 1)), (1, (2, 1)), (2, (2, 2)), (0, (2, 2)), (3, (1, 0)), (2, (2, 3)), (0, (1, 2)), (0, (1, 3)), (1, (2, 1)), (2, (1, 0)), (3, (1, 0)), (1, (2, 2)), (1, (2, 2)), (2, (1, 0)), (3, (2, 2)), (0, (2, 0)), (0, (1, 3)), (1, (2, 1)), (3, (2, 1)), (1, (2, 1)), (0, (2, 0)), (3, (2, 1)), (2, (1, 0)), (1, (2, 0)), (2, (2, 0)), (3, (2, 3)), (1, (2, 1)), (2, (2, 0)), (3, (2, 3)), (0, (1, 1)), (2, (2, 0)), (0, (2, 2)), (3, (1, 1)), (1, (1, 0)), (2, (1, 3)), (0, (1, 1)), (3, (2, 1)), (1, (2

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [31]:
import random

def RECORDREADER(count):
  data = []
  for _ in range(count):
    data.append((random.randint(0, 3), random.randint(0, 3), random.randint(0, 3)))
  return data

def MAP(t):
  return (t[0], t[2])

def REDUCE(key, values):
  return (key, sum(values)) # sum(values) - тета

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)

mapped = list(map(MAP, record))
print("MAP:", mapped)

shuffle_output = group_by_key(mapped)
shuffle_output = list(shuffle_output)
print("Shuffle output:", shuffle_output)

reduced = list(map(lambda x: REDUCE(*x), shuffle_output))
print("REDUCED:", reduced)


MAP: [(0, 0), (1, 1), (0, 2), (2, 3), (1, 3), (1, 1), (3, 0), (0, 0), (2, 1), (2, 1), (1, 0), (3, 1), (0, 2), (2, 3), (0, 3), (3, 3), (1, 0), (1, 2), (2, 0), (3, 1), (2, 2), (2, 3), (3, 1), (3, 0), (0, 1), (3, 1), (1, 0), (0, 0), (0, 0), (3, 1), (0, 1), (3, 1), (2, 2), (2, 0), (1, 3), (3, 1), (0, 2), (0, 0), (0, 2), (3, 2), (3, 2), (0, 3), (3, 3), (2, 2), (3, 0), (2, 1), (3, 2), (3, 1), (1, 2), (3, 1), (3, 0), (1, 3), (2, 2), (1, 3), (2, 1), (2, 1), (0, 2), (1, 2), (1, 0), (1, 2), (3, 3), (3, 1), (2, 2), (3, 1), (0, 3), (2, 0), (1, 2), (1, 1), (1, 2), (0, 0), (0, 3), (1, 0), (2, 1), (3, 1), (1, 1), (2, 3), (3, 0), (1, 2), (0, 0), (2, 1), (2, 2), (0, 2), (1, 2), (0, 0), (3, 1), (1, 1), (0, 2), (0, 0), (2, 3), (2, 2), (3, 2), (1, 1), (3, 0), (1, 3), (0, 1), (1, 0), (3, 0), (3, 3), (1, 0), (3, 1)]
Shuffle output: [(0, [0, 2, 0, 2, 3, 1, 0, 0, 1, 2, 0, 2, 3, 2, 3, 0, 3, 0, 2, 0, 2, 0, 1]), (1, [1, 3, 1, 0, 0, 2, 0, 3, 2, 3, 3, 2, 0, 2, 2, 1, 2, 0, 1, 2, 2, 1, 1, 3, 0, 0]), (2, [3, 1, 1, 3,

# 

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [32]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [33]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])
      
def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])
  
def REDUCE(key, values):
  (i, k) = key
  el_value = 0
  for v in values:
    el_value += v
  yield ((i, k), el_value)

Проверьте своё решение

In [34]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [35]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [36]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def RECORDREADER():
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      yield ((0, i, j), small_mat[i,j])

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((1, j, k), big_mat[j,k])

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def MAP_MUL(k1, v1):
  (i, k) = k1
  yield ((i, k), v1)

def REDUCE_MUL(key, values):
  res_el_value = 0
  for v in values:
    res_el_value += v
  yield (key, res_el_value)

def GET_JOINED():
  for j in joined:
    yield j


joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)
np.allclose(reference_solution, asmatrix(solution))

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [37]:
import numpy as np

I = 2
J = 3
K = 4*10

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j]))
  yield first_mat

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))
  yield second_mat

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 4
reducers = 2

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)

np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), np.float64(0.2006236725967179)), ((0, 1), np.float64(0.05306469128659889)), ((0, 2), np.float64(0.22647345467327412)), ((0, 3), np.float64(0.15455511727770282)), ((0, 4), np.float64(0.03770202299866222)), ((0, 5), np.float64(0.5793161664477227)), ((0, 6), np.float64(0.44320957698328267)), ((0, 7), np.float64(0.3913604851019711)), ((0, 8), np.float64(0.40770229457071205)), ((0, 9), np.float64(0.28514871845904827)), ((0, 10), np.float64(0.043711127010814685)), ((0, 11), np.float64(0.20135656075499833)), ((0, 12), np.float64(0.6410074987953122)), ((0, 13), np.float64(0.43009017858855764)), ((0, 14), np.float64(0.004661585608768176)), ((0, 15), np.float64(0.619833949719205)), ((0, 16), np.float64(0.5015960128081044)), ((0, 17), np.float64(0.09745609432018913)), ((0, 18), np.float64(0.33344309049438536)), ((0, 19), np.float64(0.5192555791955454)), ((0, 20), np.float64(0.3719400901634377)), ((0, 21), np.float64(0.457701461741589)),

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [38]:
import numpy as np

I = 2
J = 3
K = 4*10

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

def INPUTFORMAT():
  first_mat = []

  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j]))

  global maps
  split_size = int(np.ceil(len(first_mat)/maps))

  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))

  split_size = int(np.ceil(len(second_mat)/maps))

  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1

  if mat_num == 0:
    yield (j, (mat_num, i, w))

  else:
    yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]

  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0

  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []

for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)

np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), np.float64(0.12475000306175146)), ((0, 1), np.float64(0.23535709491361148)), ((0, 2), np.float64(0.2370108091066125)), ((0, 3), np.float64(0.02982928363867322)), ((0, 4), np.float64(0.16356178952853118)), ((0, 5), np.float64(0.16930900844269564)), ((0, 6), np.float64(0.13212162514196626)), ((0, 7), np.float64(0.0462647030132823)), ((0, 8), np.float64(0.17794739733428908)), ((0, 9), np.float64(0.23571699405278043)), ((0, 10), np.float64(0.17978434572247123)), ((0, 11), np.float64(0.23650001328796616)), ((0, 12), np.float64(0.2069308877961366)), ((0, 13), np.float64(0.007869609419888953)), ((0, 14), np.float64(0.047234984852700224)), ((0, 15), np.float64(0.011414267980562628)), ((0, 16), np.float64(0.17336556623703722)), ((0, 17), np.float64(0.19138881949459616)), ((0, 18), np.float64(0.17882608601957856)), ((0, 19), np.float64(0.06433997370430637)), ((0, 20), np.float64(0.11499030356879542)), ((0, 21), np.float64(0.20590106654

True